In [ ]:
import requests
import pandas as pd

In [29]:
url = "https://api-v3.balancer.fi"

query = """
{
	tokenGetTokens(
    chains: [MAINNET]
  ) {
    chain
    symbol
    underlyingTokenAddress
    address
    priceRateProviderData {
      address
      reviewed
    }
    isErc4626
    erc4626ReviewData {
      summary
    }
    priority
  }
}
"""

response = requests.post(
    url,
    json={'query': query}
)

In [4]:
tokens_raw = pd.DataFrame(response.json()['data']['tokenGetTokens'])

In [5]:
tokens_raw.head()

,chain,symbol,underlyingTokenAddress,address,priceRateProviderData,websiteUrl,isErc4626,erc4626ReviewData,priority
0,MAINNET,AAVE,None,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,None,None,False,None,0
1,MAINNET,WETH,None,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,None,False,None,0
2,MAINNET,wstETH,None,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,{'address': '0x72d07d7dca67b8a406ad1ec34ce969c...,None,False,None,0
3,MAINNET,osETH,None,0xf1c9acdc66974dfb6decb12aa385b9cd01190e38,{'address': '0x8023518b2192fb5384dadc596765b3d...,None,False,None,0
4,MAINNET,rETH,None,0xae78736cd615f374d3085123a210448e74fc6393,{'address': '0x1a8f81c256aee9c640e14bb0453ce24...,None,False,None,0


In [7]:
tokens_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   chain                   1947 non-null   object
 1   symbol                  1947 non-null   object
 2   underlyingTokenAddress  81 non-null     object
 3   address                 1947 non-null   object
 4   priceRateProviderData   101 non-null    object
 5   websiteUrl              0 non-null      object
 6   isErc4626               1947 non-null   bool  
 7   erc4626ReviewData       25 non-null     object
 8   priority                1947 non-null   int64 
dtypes: bool(1), int64(1), object(7)
memory usage: 123.7+ KB


In [6]:
tokens = tokens_raw.copy()
tokens['rateProviderAddress'] = tokens['priceRateProviderData'].apply(lambda x: x['address'] if x else None)
tokens['rateProviderReviewed'] = tokens['priceRateProviderData'].apply(lambda x: x['reviewed'] if x else None)
tokens['erc4626ReviewSummary'] = tokens['erc4626ReviewData'].apply(lambda x: x['summary'] if x else None)
tokens = tokens.drop(columns=['priceRateProviderData', 'erc4626ReviewData'])
tokens.head()

,chain,symbol,underlyingTokenAddress,address,websiteUrl,isErc4626,priority,rateProviderAddress,rateProviderReviewed,erc4626ReviewSummary
0,MAINNET,AAVE,None,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,None,False,0,None,None,None
1,MAINNET,WETH,None,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,False,0,None,None,None
2,MAINNET,wstETH,None,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,None,False,0,0x72d07d7dca67b8a406ad1ec34ce969c90bfee768,True,None
3,MAINNET,osETH,None,0xf1c9acdc66974dfb6decb12aa385b9cd01190e38,None,False,0,0x8023518b2192fb5384dadc596765b3dd1cdfe471,True,None
4,MAINNET,rETH,None,0xae78736cd615f374d3085123a210448e74fc6393,None,False,0,0x1a8f81c256aee9c640e14bb0453ce247ea0dfe6f,True,None


In [14]:
tokens[~tokens['underlyingTokenAddress'].isnull()].head()

,chain,symbol,underlyingTokenAddress,address,websiteUrl,isErc4626,priority,rateProviderAddress,rateProviderReviewed,erc4626ReviewSummary
7,MAINNET,fwstETH,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,0x2411802d8bea09be0af8fd8d08314a63e706b29c,None,True,0,0x8be2e3d4b85d05cac2dbbac6c42798fb342aef45,True,safe
8,MAINNET,waEthLidowstETH,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,0x775f661b0bd1739349b9a2a3ef60be277c5d2d29,None,True,0,0xcdaa68ce322728fe4185a60f103c194f1e2c47bc,True,safe
9,MAINNET,Aave Prime GHO,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,0xc71ea051a5f82c67adcf634c36ffe6334793d24c,None,True,0,0x851b73c4bfd5275d47fff082f9e8b4997dccb253,True,safe
10,MAINNET,waEthWETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x0bfc9d54fc184518a81162f8fb99c2eaca081202,None,True,0,0xbe7be04807762bc433911dd927fd54a385fa91d6,True,safe
12,MAINNET,fWETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x90551c1795392094fe6d29b758eccd233cfaa260,None,True,0,0x8fc43e76874cae40939edeb90e5683258b63c508,True,safe


In [16]:
# Function to define a new column called 'underlyingIsErc4626'
# It checks if the 'underlyingTokenAddress' exists in the 'address' column of the tokens DataFrame and if the corresponding 'isErc4626' is True
def check_underlying_is_erc4626(row):
    if pd.isnull(row['underlyingTokenAddress']):
        return False
    underlying_token = tokens[tokens['address'] == row['underlyingTokenAddress']]
    if not underlying_token.empty and underlying_token.iloc[0]['isErc4626']:
        return True
    return False

tokens['underlyingIsErc4626'] = tokens.apply(check_underlying_is_erc4626, axis=1)

In [18]:
tokens[['symbol', 'underlyingTokenAddress', 'underlyingIsErc4626']][tokens['underlyingIsErc4626'] == True].head(10)

,symbol,underlyingTokenAddress,underlyingIsErc4626
51,slpETH,0xa684eaf215ad323452e2b2bf6f817d4aa5c116ab,True
372,csUSDL,0x7751e2f4b8ae93ef6b79d86419d42fe3295a4559,True
607,csUSDL,0x7751e2f4b8ae93ef6b79d86419d42fe3295a4559,True


In [19]:
tokens[tokens.address == '0xa684eaf215ad323452e2b2bf6f817d4aa5c116ab']

,chain,symbol,underlyingTokenAddress,address,websiteUrl,isErc4626,priority,rateProviderAddress,rateProviderReviewed,erc4626ReviewSummary,underlyingIsErc4626
177,MAINNET,lpETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa684eaf215ad323452e2b2bf6f817d4aa5c116ab,None,True,0,None,None,None,False


In [20]:
tokens[tokens.address == '0x7751e2f4b8ae93ef6b79d86419d42fe3295a4559']

,chain,symbol,underlyingTokenAddress,address,websiteUrl,isErc4626,priority,rateProviderAddress,rateProviderReviewed,erc4626ReviewSummary,underlyingIsErc4626
1760,MAINNET,wUSDL,0xbdc7c08592ee4aa51d06c27ee23d5087d65adbcd,0x7751e2f4b8ae93ef6b79d86419d42fe3295a4559,None,True,0,0x51b7f50044a2a1a9a846575be33c00a68e0172df,True,None,False


In [24]:
tokens[['symbol', 'underlyingTokenAddress', 'underlyingIsErc4626']][tokens['underlyingIsErc4626'] == False][~tokens.underlyingTokenAddress.isnull()].head()

/var/folders/2r/rrgnpjb97p1g04q2w7rrck8c0000gn/T/ipykernel_55471/3021236057.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tokens[['symbol', 'underlyingTokenAddress', 'underlyingIsErc4626']][tokens['underlyingIsErc4626'] == False][~tokens.underlyingTokenAddress.isnull()].head()


,symbol,underlyingTokenAddress,underlyingIsErc4626
7,fwstETH,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,False
8,waEthLidowstETH,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,False
9,Aave Prime GHO,0x40d16fc0246ad3160ccc09b8d0d3a2cd28ae6c2f,False
10,waEthWETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,False
12,fWETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,False


In [27]:
tokens[tokens.address == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']

,chain,symbol,underlyingTokenAddress,address,websiteUrl,isErc4626,priority,rateProviderAddress,rateProviderReviewed,erc4626ReviewSummary,underlyingIsErc4626
1,MAINNET,WETH,None,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,None,False,0,None,None,None,False
